In [1]:
!git clone https://github.com/LC1332/Luotuo-Text-Embedding.git

Cloning into 'Luotuo-Text-Embedding'...
remote: Enumerating objects: 319, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 319 (delta 81), reused 113 (delta 46), pack-reused 168
Receiving objects: 100% (319/319), 26.39 MiB | 31.17 MiB/s, done.
Resolving deltas: 100% (179/179), done.


In [2]:
# Requirements

!pip install transformers
!pip install torch
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True, model_args=model_args)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [4]:
import csv
import numpy as np
def get_evalCSV():
    text_left = []
    text_right = []
    with open("Luotuo-Text-Embedding/data/sentspair.csv", "r") as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            text_left.append(row[0])
            text_right.append(row[1])
    return text_left, text_right

text_left, text_right = get_evalCSV()
inputs = tokenizer(text_left, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings_left = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
inputs = tokenizer(text_right, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings_right = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
    
cos_sim_matrix = torch.matmul(embeddings_left, embeddings_right.t())
cos_sim_matrix /= torch.matmul(torch.norm(embeddings_left, dim=1, keepdim=True), torch.norm(embeddings_right, dim=1, keepdim=True).t())
tensor_cpu = cos_sim_matrix.cpu()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
!pip install openai
!pip install openTSNE
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 148.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 

In [6]:
%cd Luotuo-Text-Embedding

/content/Luotuo-Text-Embedding


In [7]:
%ls

data/  image/  lib/  LICENSE  notebook/  README.md  report.md  script/


In [8]:
import sys

from lib.tsne import TSNE_Plot

merged_list = text_left + text_right
merged_embed = torch.cat((embeddings_left, embeddings_right), dim=0)

# if the data have no labels, you can use the following code to cluster the data
tsne_plot = TSNE_Plot(merged_list, merged_embed, n_clusters = 4)
tsne_plot.tsne_plot(n_sentence=40)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [10]:
%cd lib

/content/Luotuo-Text-Embedding/lib


In [11]:
import pandas as pd
from lib.heatmap import Heatmap
positions = [(i, i) for i in range(0, 20, 2)] + [(1, 5), (2, 3), (15, 9), (5, 13), (17, 7)]
df = pd.DataFrame({ "first":text_left, 
                    "second":text_right, 
                    "first_embed":[np.array(embeddings_left[i]) for i in range(len(embeddings_left))], 
                    "second_embed":[np.array(embeddings_right[i]) for i in range(len(embeddings_right))]})
heatmap = Heatmap(df, positions)
heatmap.create_heatmap()